In [ ]:
# install pycaret
!pip install pycaret

In [ ]:
# import libraries
import pandas as pd
import numpy as np

In [ ]:
# read csv data
data = pd.read_csv('../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')
data.head()

# Exploratory Data Analysis

In [ ]:
# check data types
data.dtypes

In [ ]:
# replace blanks with np.nan
data['TotalCharges'] = data['TotalCharges'].replace(' ', np.nan)
data.isnull().sum()

In [ ]:
# convert to float64
data['TotalCharges'] = data['TotalCharges'].astype('float64')
data.info()

In [ ]:
!pip install pandas-profiling

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(data, title="EDA Report")
profile

# Missing Values

In [ ]:
# check missing values
data.isnull().sum()

In [ ]:
round(data.Churn.value_counts()*100 / len(data),2)

In [ ]:
categorical = []
for i in data.columns:
    if (data[i].dtype=='object'):
        categorical.append(i)
print("Categorical Attribute : {}\n ".format(len(categorical)))
for x in range(len(categorical)): 
    print(categorical[x])

In [ ]:
data[categorical].nunique()

In [ ]:
for i in categorical[1:]:
    print(i)
    print(data[i].unique())
    print("\n")

# Data Preparation

In [ ]:
# init setup
from pycaret.classification import *
telecom = setup(data, target = 'Churn', ignore_features = ['customerID'],
                ordinal_features = {'Contract' : ['Month-to-month' ,'One year', 'Two year']},
                fix_imbalance = True,
               transformation = True,#Transformation changes the shape of the distribution such that the transformed data can be represented by normal distribution
                  normalize = True, #rescale the values of numeric columns
                  handle_unknown_categorical = True, 
                  unknown_categorical_method = 'most_frequent',
                  remove_multicollinearity = True, #rop one of the two features that are highly correlated with each other
                  ignore_low_variance = True,#all categorical features with statistically insignificant variances are removed from the dataset.
                  combine_rare_levels = True,# all levels in categorical features below the threshold defined in rare_level_threshold param are combined together as a single level
                numeric_imputation='median',
                categorical_imputation='mode',)

In [ ]:
catboost= create_model('catboost')

In [ ]:
print(catboost)

In [ ]:
tuned_catboost = tune_model(catboost,optimize = 'Accuracy') #tuned on Accuracy

In [ ]:
evaluate_model(tuned_catboost) #Graphical plot 

In [ ]:
predict_model(tuned_catboost)# Test data evaluation 

In [ ]:
final_catboost = finalize_model(tuned_catboost) # Final model 
final_catboost

In [ ]:
predict_model(final_catboost)#final test model evaluation 